# 0. Dependencies & Setup

## 0.1. Related dependencies and environment variables

In [1]:
# Install spark-related dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://mirrors.viethosting.com/apache/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark

# Set up required environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"

     |████████████████████████████████| 215.7MB 62kB/s 
     |████████████████████████████████| 204kB 37.9MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=ee7f2262fa69938a63b30ef25d70732284eb8cde7cf37092f6274c450052c9b4
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


## 0.2. Initialize Spark

In [2]:
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')
 
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

spark = SparkSession \
.builder \
.appName("sparkml_lab") \
.master("local[*]") \
.getOrCreate()

# Check
spark.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1574296037224'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'sparkml_lab'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '37103'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', '9cade6b8ecb1')]

In [3]:
spark

# 1. Import Housing Dataset

In [4]:
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()

TypeError: ignored

# 2. Data Discovery

In [0]:
housing = spark.read.csv("/content/datasets/housing/housing.csv", header = True)

## 2.1. Schema and dimensions

Printing schema of the dataset

In [8]:
housing.printSchema()

root
 |-- longitude: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- housing_median_age: string (nullable = true)
 |-- total_rooms: string (nullable = true)
 |-- total_bedrooms: string (nullable = true)
 |-- population: string (nullable = true)
 |-- households: string (nullable = true)
 |-- median_income: string (nullable = true)
 |-- median_house_value: string (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Printing number of records in the dataset

In [10]:
housing.count()

20640

## 2.2. Look at the data

Printing first five records of the dataset

In [11]:
housing.take(5)

[Row(longitude='-122.23', latitude='37.88', housing_median_age='41.0', total_rooms='880.0', total_bedrooms='129.0', population='322.0', households='126.0', median_income='8.3252', median_house_value='452600.0', ocean_proximity='NEAR BAY'),
 Row(longitude='-122.22', latitude='37.86', housing_median_age='21.0', total_rooms='7099.0', total_bedrooms='1106.0', population='2401.0', households='1138.0', median_income='8.3014', median_house_value='358500.0', ocean_proximity='NEAR BAY'),
 Row(longitude='-122.24', latitude='37.85', housing_median_age='52.0', total_rooms='1467.0', total_bedrooms='190.0', population='496.0', households='177.0', median_income='7.2574', median_house_value='352100.0', ocean_proximity='NEAR BAY'),
 Row(longitude='-122.25', latitude='37.85', housing_median_age='52.0', total_rooms='1274.0', total_bedrooms='235.0', population='558.0', households='219.0', median_income='5.6431', median_house_value='341300.0', ocean_proximity='NEAR BAY'),
 Row(longitude='-122.25', latitude

Printing the number of records with population more than 10000

In [56]:
housing.where(housing.population > 10000).collect()

[Row(longitude='-121.92', latitude='37.53', housing_median_age='7.0', total_rooms='28258.0', total_bedrooms='3864.0', population='12203.0', households='3701.0', median_income='8.4045', median_house_value='451100.0', ocean_proximity='<1H OCEAN'),
 Row(longitude='-117.78', latitude='34.03', housing_median_age='8.0', total_rooms='32054.0', total_bedrooms='5290.0', population='15507.0', households='5050.0', median_income='6.0191', median_house_value='253900.0', ocean_proximity='<1H OCEAN'),
 Row(longitude='-117.87', latitude='34.04', housing_median_age='7.0', total_rooms='27700.0', total_bedrooms='4179.0', population='15037.0', households='4072.0', median_income='6.6288', median_house_value='339700.0', ocean_proximity='<1H OCEAN'),
 Row(longitude='-117.88', latitude='33.96', housing_median_age='16.0', total_rooms='19059.0', total_bedrooms='3079.0', population='10988.0', households='3061.0', median_income='5.5469', median_house_value='265200.0', ocean_proximity='<1H OCEAN'),
 Row(longitude=

## 2.3. Statistical summary

Printing summary of the table statistics for the attributes housing_median_age, total_rooms, median_house_value, and population.

In [16]:
housing.describe(["housing_median_age", "total_rooms", "median_house_value", "population"]).show()

+-------+------------------+------------------+------------------+------------------+
|summary|housing_median_age|       total_rooms|median_house_value|        population|
+-------+------------------+------------------+------------------+------------------+
|  count|             20640|             20640|             20640|             20640|
|   mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
| stddev| 12.58555761211163|2181.6152515827944|115395.61587441359|  1132.46212176534|
|    min|               1.0|             100.0|          100000.0|             100.0|
|    max|               9.0|            9998.0|           99900.0|             999.0|
+-------+------------------+------------------+------------------+------------------+



Print the maximum age (housing_median_age), the minimum number of rooms (total_rooms), and the average of house values (median_house_value).

In [55]:
value1 = housing.agg({"housing_median_age": "max"}).collect()[0]
value2 = housing.agg({"total_rooms" : "min"}).collect()[0]
value3 = housing.agg({"median_house_value" : "mean"}).collect()[0]

print("Maximum housing age: " + str(value1["max(housing_median_age)"]) + 
      "\nMinimum number of rooms: " + str(value2["min(total_rooms)"]) +
      "\nAverage house value: " + str(value3["avg(median_house_value)"]))

Maximum housing age: 9.0
Minimum number of rooms: 100.0
Average house value: 206855.81690891474


## 2.4. Data breakdown by categorical data

Printing the number of houses in different areas (ocean_proximity), and sorted in descending order.

In [74]:
housing.cube("ocean_proximity").count().sort('count', ascending=False).show()

+---------------+-----+
|ocean_proximity|count|
+---------------+-----+
|           null|20640|
|      <1H OCEAN| 9136|
|         INLAND| 6551|
|     NEAR OCEAN| 2658|
|       NEAR BAY| 2290|
|         ISLAND|    5|
+---------------+-----+



Printing the average value of the houses (median_house_value) in different areas (ocean_proximity); calling the new column avg_value when printing it.

In [117]:
from pyspark.sql import functions as F

housing.\
select(["ocean_proximity", "median_house_value"]).\
groupBy("ocean_proximity").\
agg(F.mean("median_house_value").alias("avg_value")).\
show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



Rewritten in SQL.

In [121]:
housing.createOrReplaceTempView("housing")
spark.sql('''
          SELECT ocean_proximity, avg(median_house_value)
          AS avg_value
          FROM housing
          GROUP BY ocean_proximity
          ''').show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



## 2.5. Correlation among attributes

Printing the correlation among the attributes housing_median_age, total_rooms, median_house_value, and population. To do so, first we need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, we use the VectorAssembler Transformer.

## 2.6. Combine and make new attributes

Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, we add the three new columns to the dataset as below. We will call the new dataset the housingExtra.

```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```



# 3. Data Preparation

To be continued ;)